# Example State Machine Usage
## Inter-Machine Communication

In this example, suppose we have two state machines, representing a waiter and cook.

The waiter:

 - Takes customer order
 - Enters Order
 - Waits for Order
 - Gives order to customer
 
The cook:

 - Waits for an order
 - Prepares the order
 - Puts order in "the pass"
 - Waits for another order




In [1]:
from state_machine import StateMachine, State, Condition, AlwaysTrue, AlwaysFalse, Event

In [2]:
class CustomerWaves(Condition):
    listens_for='customer'
    
    def __init__(self):
        super(CustomerWaves,self).__init__(name='Customer Waves')
    
    def is_triggered(self, event):
        return event.payload=='waves'

CUSTOMER_WAVES = CustomerWaves()

In [3]:
class CustomerOrders(Condition):
    listens_for='customer'
    
    def __init__(self):
        super(CustomerOrders,self).__init__(name='Customer Orders')
        
    def is_triggered(self, event):
        return 'orders' in event.payload
            
CUSTOMER_ORDERS = CustomerOrders()

In [4]:
class WhenOrderComplete(Condition):
    listens_for='order_complete'
    
    def __init__(self):
        super(WhenOrderComplete,self).__init__(name='When Order Complete')
        
    def is_triggered(self, event):
        return True
    
WHEN_ORDER_COMPLETE = WhenOrderComplete()

### States

In [5]:
class TakeCustomerOrder(State):
        
    def on_start(self, event):
        order = Event('order', event.payload)
        return order

In [6]:
class EnterCustomerOrder(State):
    
    def on_start(self, event):
        evt = Event( 'order_entered', event.payload['orders'] )
        self.logger.info( 'Sending order_entered event' )
        self.notify_observers(evt)
        return evt

In [7]:
class GiveOrderToCustomer(State):
    
    def on_start(self, event):
        completed_order = event.payload
        self.logger.info( 'Gives %s to customer', completed_order)

In [8]:
waiter = StateMachine('waiter')

state_loiter                     = waiter.create_state('Loitering')
state_wait_for_customer_to_order = waiter.create_state('Wait For Customer to Order')
state_take_customer_order        = waiter.create_state( 'Take Customer Order', TakeCustomerOrder)
state_enter_customer_order       = waiter.create_state( 'Enter Customer Order', EnterCustomerOrder)
state_wait_for_order             = waiter.create_state('Wait For Order')
state_give_order_to_customer     = waiter.create_state( 'Give Order to Customer', GiveOrderToCustomer)

state_loiter.add_transition_to(state_wait_for_customer_to_order, CUSTOMER_WAVES)
state_wait_for_customer_to_order.add_transition_to(state_take_customer_order, CUSTOMER_ORDERS)
state_take_customer_order.add_default_transition_to(state_enter_customer_order)
state_enter_customer_order.add_default_transition_to(state_wait_for_order)
state_wait_for_order.add_transition_to(state_give_order_to_customer,WHEN_ORDER_COMPLETE)

In [9]:
waiter.set_state(state_loiter)

e = Event('customer','waves')
waiter.logger.info('Customer waves')
waiter.notify(e)

e = Event('customer',{'orders':'pizza'})
waiter.logger.info( 'Customer orders pizza' )
waiter.notify(e)

# Simulating the result from the chef
e = Event('order_complete','cooked_pizza')
waiter.logger.info( 'Giving waiter a completed order' )
waiter.notify(e)

INFO:default:Customer waves
INFO:default:Starting state Wait For Customer to Order
INFO:default:Wait For Customer to Order started successfully returning event: None
INFO:default:Customer orders pizza
INFO:default:Starting state Take Customer Order
INFO:default:Take Customer Order started successfully returning event: <Event name="order", payload={'orders': 'pizza'}>
INFO:default:Starting state Enter Customer Order
INFO:default:Sending order_entered event
INFO:default:Enter Customer Order started successfully returning event: <Event name="order_entered", payload=pizza>
INFO:default:Starting state Wait For Order
INFO:default:Wait For Order started successfully returning event: None
INFO:default:Giving waiter a completed order
INFO:default:Starting state Give Order to Customer
INFO:default:Gives cooked_pizza to customer
INFO:default:Give Order to Customer started successfully returning event: None


In [10]:
class WhenOrderEntered(Condition):
    listens_for='order_entered'
    
    def __init__(self):
        super(WhenOrderEntered,self).__init__(name='When Order Entered')
        
    def is_triggered(self, event):
        print( 'WhenOrderEntered received %s'%event)
        return True
    
WHEN_ORDER_ENTERED = WhenOrderEntered()

In [21]:
class WhenOrderPrepared(Condition):
    listens_for='time_passes'
    
    def __init__(self):
        super(WhenOrderPrepared,self).__init__(name='When Order Entered')
        
    def is_triggered(self, event):
        return True
    
WHEN_ORDER_PREPARED = WhenOrderPrepared()

In [25]:
class ChefPreparesOrder(State):
    def on_start(self, event):
        self.logger.info( 'Chef starts preparing %s'%event.payload)

In [26]:
class ChefCompletesOrder(State):
    
    def on_start(self, event):
        self.logger.info('Chef has completed order')
        e = Event('order_complete','cooked_pizza')
        self.notify_observers(e)

In [27]:
chef = StateMachine('chef')
chef_waits_for_order = chef.create_state('Chef Waits For Order')
chef_prepares_order  = chef.create_state('Chef Prepares Order', ChefPreparesOrder)
chef_completes_order = chef.create_state('Chef Completes Order', ChefCompletesOrder)

chef_waits_for_order.add_transition_to(chef_prepares_order, WHEN_ORDER_ENTERED)
chef_prepares_order.add_transition_to(chef_completes_order, WHEN_ORDER_PREPARED)

In [28]:
chef.set_state(chef_waits_for_order)

e = Event('order_entered','pizza')
chef.notify(e)

chef.logger.info( chef.current_state.start_event)

e = Event('time_passes','10 minutes')
chef.notify(e)

INFO:default:Starting state Chef Prepares Order
INFO:default:Chef starts preparing pizza
INFO:default:Chef Prepares Order started successfully returning event: None
INFO:default:<Event name="order_entered", payload=pizza>
INFO:default:Starting state Chef Completes Order
INFO:default:Chef has completed order
INFO:default:Chef Completes Order started successfully returning event: None


WhenOrderEntered received <Event name="order_entered", payload=pizza>


In [15]:
chef

In [16]:
waiter.set_state(state_loiter)
waiter.register_observer('order_entered', chef)

chef.set_state(chef_waits_for_order)

e = Event('customer','waves')
waiter.logger.info('Customer waves')
waiter.notify(e)

e = Event('customer',{'orders':'pizza'})
waiter.logger.info( 'Customer orders pizza' )
waiter.notify(e)

e = Event('time_passes','10 minutes')
chef.logger.info( 'time passing...' )
chef.notify(e)

INFO:default:Customer waves
INFO:default:Starting state Wait For Customer to Order
INFO:default:Wait For Customer to Order started successfully returning event: None
INFO:default:Customer orders pizza
INFO:default:Starting state Take Customer Order
INFO:default:Take Customer Order started successfully returning event: <Event name="order", payload={'orders': 'pizza'}>
INFO:default:Starting state Enter Customer Order
INFO:default:Sending order_entered event
INFO:default:Starting state Chef Prepares Order
INFO:default:Chef Prepares Order started successfully returning event: None
INFO:default:Enter Customer Order started successfully returning event: <Event name="order_entered", payload=pizza>
INFO:default:Starting state Wait For Order
INFO:default:Wait For Order started successfully returning event: None
INFO:default:time passing...
INFO:default:Starting state Chef Completes Order
INFO:default:Chef has completed order
INFO:default:Chef Completes Order started successfully returning event

WhenOrderEntered received <Event name="order_entered", payload=pizza>


In [17]:
WhaT IS the chef preparing?

Object `preparing` not found.


In [ ]:
WhaT IS the chef preparing

In [18]:
WhaT IS the chef preparing

SyntaxError: invalid syntax (<ipython-input-18-73a0de0905e2>, line 1)